# Modelos y Simulación 2024

## Trabajo Práctico Final:
## Hospital

### Alumnos:
- Ana Enzo
- Barea Matias
- Irigoyen Carlos
- Morillo Alen

### Profesores: Lic. Claudia López de Munain - Lic. Bruno Alfredo Pazos

In [1]:
from hospital import *

In [3]:
''' TEST SIMULACION'''
estado_sistema = simulacion(2,210,4,12,130,4)
print(estado_sistema.keys())
print(f'Cantidad de quirofanos:', estado_sistema['cantidad_de_quirofanos'],'\n',
        'Kits iniciales:', estado_sistema['kits_iniciales'], '\n',
        'Kits disponibles:', estado_sistema['kits_diarios_disponibles'], '\n',
        'kits utilizados por dia:', estado_sistema['kits_diarios_utilizados'], '\n',
        'llegada de pacientes por dia:', estado_sistema['llegadas_por_dia'], '\n',
        'reservas de quirofano por dia:', estado_sistema['reservas_por_dia'], '\n',
        'disponibilidad de camas por dia:', estado_sistema['disponibilidad_camas'], '\n',
        'ocupacion diaria quirofanos (en horas):', estado_sistema['ocupacion_diaria_quirofanos'], '\n',
        'cuanto esperan los pacientes por dia en ser operados:', estado_sistema['tiempo_espera_diario_quirofanos'] , '\n',
        'pacientes que solo se internaron:', estado_sistema['pacientes_sin_reserva_por_dia'], '\n',
        'pacientes rechazados por dia por ausencia de  camas : ', estado_sistema['pacientes_rechazados_por_ausencia_camas'], '\n',
        'pacientes rechazados por dia por internacion agotada : ', estado_sistema['pacientes_rechazados_por_internacion_agotada'], '\n'
        'cirugias concretadas por dia: ', estado_sistema['cirugias_concretadas_por_dia'], '\n',
        'cirugias rechazadas por dia: ', estado_sistema['cirugias_rechazadas_por_dia'], '\n',
        'cirugias diarias reprogramadas por falta de insumos: ', estado_sistema['cirugias_reprogramadas_por_insumos'], '\n',
        'cirugias diaria reprogramadas al otro dia por ocupacion del quirofano:', estado_sistema['cirugias_reprogramadas_por_tiempo'], '\n',
        'cirugias diaria reprogramadas al otro dia por cuota diria: ', estado_sistema['cirugias_reprogramadas_por_cuota_diaria'], '\n',
        'Total de cirugias reprogramadas por falta insumos: ', estado_sistema['total_cirugias_reprogramadas_por_insumos'], '\n',
        'Total de cirugias reprogramadas por falta de tiempo: ', estado_sistema['total_cirugias_reprogramadas_por_tiempo'], '\n',
        'Total de cirugias reprogramadas por cuota diaria: ', estado_sistema['total_cirugias_reprogramadas_por_cuota_diaria'], '\n',
        'Total cirugias concretadas: ', estado_sistema['total_cirugias_concretadas'], '\n',
        'Total cirugias rechazadas: ', estado_sistema['total_cirugias_rechazadas'], '\n',
        'Total pacientes rechazados por ausencia de camas: ', estado_sistema['total_pacientes_rechazados_por_ausencia_camas'], '\n',
        'Total pacientes rechazados por internacion agotada: ', estado_sistema['total_pacientes_rechazados_por_internacion_agotada'], '\n'
        'Promedio de espera de los pacientes: ', np.mean(estado_sistema['tiempo_espera_diario_quirofanos']), '\n',
        'Tiempo promedio de espera diario de los pacientes: ', (estado_sistema['tiempo_espera_diario_quirofanos']), '\n'


        )




dict_keys(['dias_simulacion', 'cantidad_de_quirofanos', 'kits_iniciales', 'llegadas_por_dia', 'reservas_por_dia', 'disponibilidad_camas', 'pacientes_rechazados_por_dia', 'pacientes_rechazados_por_ausencia_camas', 'pacientes_rechazados_por_internacion_agotada', 'pacientes_sin_reserva_por_dia', 'kits_diarios_disponibles', 'kits_diarios_utilizados', 'ocupacion_diaria_quirofanos', 'tiempo_espera_diario_quirofanos', 'cirugias_rechazadas_por_dia', 'cirugias_concretadas_por_dia', 'cirugias_reprogramadas_por_insumos', 'cirugias_reprogramadas_por_tiempo', 'cirugias_reprogramadas_por_cuota_diaria', 'total_cirugias_reprogramadas_por_insumos', 'total_cirugias_reprogramadas_por_tiempo', 'total_cirugias_reprogramadas_por_cuota_diaria', 'total_cirugias_concretadas', 'total_cirugias_rechazadas', 'total_pacientes_rechazados_por_ausencia_camas', 'total_pacientes_rechazados_por_internacion_agotada', 'tiempo_espera_promedio_de_quirofanos'])
Cantidad de quirofanos: 4 
 Kits iniciales: 130 
 Kits disponible